# Concatenating, Merging, and Compare

Pandas provides various facilities for easily combining together Series or DataFrame with various kinds of set logic.

In addition, pandas also provides utilities to compare two Series or DataFrame and summarize their differences.

Topic Reviewed

- Concatenation (vertical and horizontal) `concat()`
- Merging `merge()` and `join()`
- Comparison `compare()`

In [42]:
import pandas as pd
import numpy as np

np.random.seed(0)

In [43]:
# Handy Functions
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def letter_list(letter, length, start = None):
    if start:
        result = [f"{letter}{i}" for i in range(start, length + start)]
    else:
        result = [f"{letter}{i}" for i in range(length)]
    return result


## Concatenation: `concat()`

The function `concat()` is used to concat two (or more) DataFrames or Series
along an axis, and additionaly perform optional set logic.

`concat(objs, axis, join, ignore_index, keys)`

- `objs`: list of objects (`DataFrame` or `Series`) or dict of objects
- `axis`: the axis to concatenate along (default 0)
- `join`: set logic to define how to handle **the other axis**.
    - `join = outer` performs an union (default and zero information loss)
    - `join = inner` perform an interception
- `ignore_index` (default False):  If True, do not use the index values **on the concatenation axis**. The resulting axis will be the default. Note the index values on the other axes are still respected in the join.
- `keys`: construct hierarchical index using the passed keys as the outermost level. It is useful to indentify the source of each chunk of data in the resulting `DataFrame` (or `Series`)

**NOTE**: Take in mind that `concat()` **makes a full copy of the data**, and that constantly reusing this function can create a significant performance hit. If you need to use the operation over several datasets, use a list comprehension.

Things to take into account about `concat()`

1. Vertical and Horizontal Stacking (`axis`)
2. Addressing Duplicate index (`keys` and `ignoring_index`)
3. Difference between `join='outer'` and `join='inner'`
4. Concatenating with mixed dimensions

### Vertical and Horizontal Stacking 

To control the way to stack, you can use `axis` parameter

- Vertical Stacking with `axis = 0` (default)
- Horizontal Stacking with `axis = 1`

**NOTE**: if there are repeated labels along the axis. All the labels in that axis will appear on the result although the result will have duplicate labels in that axis.

<img src="./assets/imgs/concat_h_v_stacking.png" width="600"/>

In [44]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[0, 1, 2, 3],
)
display_side_by_side(df1, df2)


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


In [45]:
# 1. By default concat along the axis = 0
# NOTE: Although there are duplicate index labels, they are kept.
pd.concat([df1, df2])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


In [46]:
# 2. Concat along the axis = 1
# NOTE: Although there are duplicate column (axis = 1) labels, they are kept.
pd.concat([df1, df2], axis=1)

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,A3,B3,C3,D3,A7,B7,C7,D7


### Addressing Duplicate index (`keys` (or dict) and `ignoring_index`)

There are two ways to address duplicated labels:

1. The `keys = <list of identifiers>` can be used to create a multi-index that identifies each group (chunk of joined data) in the resulting `DataFrame` (or `Series`).
2. The `ignoring_index = True` instead will reset and drop the labels, and place the default labels instead `0, 1, ...`. It is commonly used when you have objects with no meaningful labels.

**NOTE:** You can get the same result of `keys` using a dictionary as input to `objs` (first parameter).

**NOTE:** `keys` and `ignoring_index` will have effect on `index` or `columns` depending what the `axis` indicates.

In [47]:
# 1. Construct multi-index to identify chunks of data using keys
# NOTE: you can index result.loc["df1"] to get chunk of data
previous = pd.concat([df1, df2])
result = pd.concat([df1, df2], keys=["df1", "df2","df3"])
display_side_by_side(previous, result)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 0 
 A4 
 B4 
 C4 
 D4 
 
 
 1 
 A5 
 B5 
 C5 
 D5 
 
 
 2 
 A6 
 B6 
 C6 
 D6 
 
 
 3 
 A7 
 B7 
 C7 
 D7 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 
 
 
 
 df1 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 df2 
 0 
 A4 
 B4 
 C4 
 D4 
 
 
 1 
 A5 
 B5 
 C5 
 D5 
 
 
 2 
 A6 
 B6 
 C6 
 D6 
 
 
 3 
 A7 
 B7 
 C7 
 D7

In [48]:
#NOTE: you can also use a dict to get the same behavior
pd.concat({"df1": df1, "df2": df2})

A   B   C   D
df1 0  A0  B0  C0  D0
    1  A1  B1  C1  D1
    2  A2  B2  C2  D2
    3  A3  B3  C3  D3
df2 0  A4  B4  C4  D4
    1  A5  B5  C5  D5
    2  A6  B6  C6  D6
    3  A7  B7  C7  D7

In [49]:
# 2. Ignoring index will reset and drop labels
previous = pd.concat([df1, df2])
result = pd.concat([df1, df2], ignore_index=True)
display_side_by_side(previous, result)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
,A,B,C,D
0,A0,B0,C0,D0


###  `join='outer'` and `join='inner'` difference

`join` attribute controls how to handle **the other axis** (other than the one being concatenated).

- `join = outer` (default) will take the **union** of the other axis. It has zero information loss.
- `join = inner` will take the **interception** of the other axis.

**NOTE:** the normal vertical or horizonal stacking will happen in the main axis (using the `axis` attribute), while the `join` will only control `union` or `interception` in the other axis. e.g: if we define the main `axis = 0` then the `other axis = 1`.

**NOTE:** if some values are missing when use `join = outer` (union), they will be replaced by NaN.

<img src="./assets/imgs/concat_join_attribute.png" width="600"/>

In [50]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[2, 3, 4, 5],
)

display_side_by_side(df1, df2)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 
 
 
 
 B 
 C 
 D 
 
 
 
 
 2 
 B4 
 C4 
 D4 
 
 
 3 
 B5 
 C5 
 D5 
 
 
 4 
 B6 
 C6 
 D6 
 
 
 5 
 B7 
 C7 
 D7

In [51]:
# vertical stacking along axis = 0
# and join = 'outer' (union default) along axis = 1
# NOTE: the rows labels are repeated because is perfomming normal vertical stacking
# but the missing values are replaced with NaN to complete the union
outer_df = pd.concat([df1, df2], axis = 0, join = 'outer')

# vertical stacking along axis = 0
# and join = 'inner' (interception) along axis = 1 (columns)
inner_df = pd.concat([df1, df2], axis = 0, join = 'inner')

display_side_by_side(outer_df, inner_df)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 2 
 NaN 
 B4 
 C4 
 D4 
 
 
 3 
 NaN 
 B5 
 C5 
 D5 
 
 
 4 
 NaN 
 B6 
 C6 
 D6 
 
 
 5 
 NaN 
 B7 
 C7 
 D7 
 
 
 
 
 
 
 B 
 C 
 D 
 
 
 
 
 0 
 B0 
 C0 
 D0 
 
 
 1 
 B1 
 C1 
 D1 
 
 
 2 
 B2 
 C2 
 D2 
 
 
 3 
 B3 
 C3 
 D3 
 
 
 2 
 B4 
 C4 
 D4 
 
 
 3 
 B5 
 C5 
 D5 
 
 
 4 
 B6 
 C6 
 D6 
 
 
 5 
 B7 
 C7 
 D7

In [52]:
# horizontal stacking along axis = 1
# and join = 'outer' (union default) along axis = 0
outer_df = pd.concat([df1, df2], axis = 1, join = 'outer')

# horizontal stacking along axis = 1
# and join = 'inner' (interception) along axis = 0 (rows)
inner_df = pd.concat([df1, df2], axis = 1, join = 'inner')

display_side_by_side(outer_df, inner_df)

,A,B,C,D,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B4,C4,D4
3,A3,B3,C3,D3,B5,C5,D5
4,NaN,NaN,NaN,NaN,B6,C6,D6
5,NaN,NaN,NaN,NaN,B7,C7,D7
,A,B,C,D,B,C,D
2,A2,B2,C2,D2,B4,C4,D4
3,A3,B3,C3,D3,B5,C5,D5


### Concatenating with mixed dimns

It is also possible to concatenate combination of `Series` and/or `DataFrame`.

1. Appending columns can be done directly by concatenating directly the `DataFrame` and `Series`. e.g: `pd.concat([df1, s1], axis = 1)`.
2. However, for appending rows, it is neecesary to convert the series to `DataFrame` and transpose. e.g: `pd.concat([df1, s1.to_frame().T])`

**NOTE:** The name of a `Series` will be converted in the name of the column (or row) in the resulting
`DataFrame`. Also, you need to have a match on the `index` of the `Series` with the respective labels of the `DataFrame`.

**NOTE:** If `Series` has no name, a default name will be assigned. Also, if you concatenate only
series, you can rename the columns using the `keys` attribute.

**NOTE:** `concat()` helps us to concatenate several cols or rows at once, but if you only want to add **one** you can directly use accessors to add them.

In [53]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

In [54]:
# 1. Concatenating 1 or 2 Series as new columns (axis = 1) to a DataFrame
s1 = pd.Series( ["F0", "F1", "F2", "F3"], name="F")
s2 = pd.Series( ["G0", "G1", "G2", "G3"], name="G")

result1 = pd.concat([df1, s1], axis=1)
result2 = pd.concat([df1, s1, s2], axis=1)

display(df1)
display(result1)
display(result2)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D,F
0,A0,B0,C0,D0,F0
1,A1,B1,C1,D1,F1
2,A2,B2,C2,D2,F2
3,A3,B3,C3,D3,F3


,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A1,B1,C1,D1,F1,G1
2,A2,B2,C2,D2,F2,G2
3,A3,B3,C3,D3,F3,G3


In [55]:
# 2. Concatenating 1 or 2 Series as new rows to a DataFrame
# NOTE: you need to match the index with the columns labels
s1 = pd.Series( ["A5", "B5", "C5", "D5"], index=["A","B","C","D"], name = 5)
s2 = pd.Series( ["A7", "B7", "C7", "D7"], index=["A","B","C","D"], name = 7)

result1 = pd.concat([df1, s1.to_frame().T])
result2 = pd.concat([df1, s1.to_frame().T, s2.to_frame().T])

display_side_by_side(df1, result1, result2)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
5,A5,B5,C5,D5


In [56]:
# Renaming a concatenation of series using keys
s1 = pd.Series( ["E0", "E1", "E2", "E3"], name="E")
s2 = pd.Series( ["F0", "F1", "F2", "F3"])
s3 = pd.Series( ["G0", "G1", "G2", "G3"])

result1 = pd.concat([s1, s2, s3], axis=1)
result2 = pd.concat([s1, s2, s3], axis=1, keys=["E","F","G"])
display_side_by_side(result1, result2)

,E,0,1
0,E0,F0,G0
1,E1,F1,G1
2,E2,F2,G2
3,E3,F3,G3
,E,F,G
0,E0,F0,G0
1,E1,F1,G1
2,E2,F2,G2
3,E3,F3,G3


## Database-style DataFrame (`join()` and `merge()`)
Pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL.

Mainly, we will use `merge()` and we see later that `join()` is a specific case of `merge()`.

## Merging: `merge()`

Merging can performs different joining operations (like SQL): one-to-one, many-to-one,
many-to-many. You can perform these operations using:

```
merge(
    left,
    right,
    how = 'inner',
    on = None,
    left_on = None,
    right_on = None,
    left_index = False,
    right_index = False
)
```

- `left` and `right` are two DataFrame or named Series objects.
- `how` controls which index labels to keep in the result. It can be `left`, `right`, `outer`, `inner`, `cross`.
- `on` defines column or index level names to join on. Must be found in both the `left` and `right` objects. If not passed and `left_index` and `right_index` are False, the intersection of the columns in the two objects will be inferred to be the join keys.
- `left_on`: Columns or index levels from the left object. Can either be column names, index level names, or arrays with length equal to the length of the left object.
- `right_on`: Same usage as `left_on` for the right object.
- `left_index`: If True, use the index (row labels) from the left object as its join key(s). In the case of an object with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right object.
- `right_index`: Same usage as `left_index` for the right DataFrame or Series

**NOTE:** You can use `merge` as `pd.merge(df1, df2)` or `df1.merge(df2)`

Things to take into account about `merge()`:

- General Idea: joining operations and set logic (`how` attribute)
    - One-to-one Operation and `how` Variants
    - One-to-Many Operation and `how` Variants.
    - Many-to-Many Operation (cross combination) and `how` Variants
- Overlapping Value Columns
- Non-matching Names on Keys
- Multiple, Index, and Index-Columns Keys
- Extras: `indicator` and `validation` arguments

### General Idea: Joining Operations and Set Logic

The general idea of `merge()` is to have two objects **left** and **right**, which 
have some common column (or index level) in order to perform a joining operation.
The common column (or index level) is known as `key`.

<img src="./assets/imgs/merge_explanation.png" width="600"/>

Based on this idea,
- the  **joining operations** are defined by the common column (key), and 
- the **set logic** are defined by the `how` attribute, which controls the keys 
that will appear in the result.

Possible Joining Operations:

1. one-to-one
2. one-to-many
3. many-to-many

Possible Set Logic:

| 'how' method  | SQL Join Name   | Description                                |
|---------------|---------------|--------------------------------------------|
| `inner`         | `INNER JOIN`     | Use **intersection** of keys from both frames   |
| `outer`         | `FULL OUTER JOIN` | Use **union** of keys from both frames          |
| `left`          | `LEFT OUTER JOIN` | Use keys from left frame only               |
| `right`         | `RIGHT OUTER JOIN` | Use keys from right frame only              |
| `cross`         | `CROSS JOIN`     | Create the cartesian product of rows of both frames          |

**NOTE:** when some data is missing in outer joins (`outer`, `left` or `right`), the spaces are filled with NaNs.

**NOTE:** when using `merge()` the `index` labels are ignored and replaced with a default `index` in the result.

**NOTE:** that `how=cross` is an special case that doesn't require to use keys. In other words, it is not required to use `on`.

#### One to One Operation and Set Logic Variants

The **one-to-one operation** `(1:1)` occurs when one row from left can only be linked 
to one row on right (linked through the key).

In other words, both left and right key values have to be **unique**. In 
this cases, the information of a same observation (row) is divided between 
the left and right objects. Then, the objective of the one-to-one operation is 
to merge this information together into a single DataFrame.

*e.g*: one user with name and lastname and the same user with phone and birth date information.

<img src="./assets/imgs/merge_one_to_one.png" width="600"/>

**NOTE:** The rows 2, 3 from left and rows 0, 1 from right share the information about the observation `B2` and `B3`.

In [57]:
left = pd.DataFrame(
    {
        "A": letter_list("A",4),
        "B": letter_list("B",4),
        "C": letter_list("C",4),
        "D": letter_list("D",4)
    }
)
right = pd.DataFrame(
    {
        "B": letter_list("B",4, start = 2),
        "F": letter_list("F",4, start = 2),
        "G": letter_list("G",4, start = 2)
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 
 
 
 
 B 
 F 
 G 
 
 
 
 
 0 
 B2 
 F2 
 G2 
 
 
 1 
 B3 
 F3 
 G3 
 
 
 2 
 B4 
 F4 
 G4 
 
 
 3 
 B5 
 F5 
 G5

In [58]:
# one to one with how = "inner" (interception of left and right keys)
pd.merge(left, right, on="B", how="inner")

,A,B,C,D,F,G
0,A2,B2,C2,D2,F2,G2
1,A3,B3,C3,D3,F3,G3


In [59]:
# one to one with how = "outer" (union of left and right keys)
# NOTE: filling missing values with NaN
outer_df = pd.merge(left, right, on="B", how="outer")
left_outer = pd.merge(left, right, on="B", how="left")
right_outer = pd.merge(left, right, on="B", how="right")

display_side_by_side(outer_df, left_outer, right_outer)

,A,B,C,D,F,G
0,A0,B0,C0,D0,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN
2,A2,B2,C2,D2,F2,G2
3,A3,B3,C3,D3,F3,G3
4,NaN,B4,NaN,NaN,F4,G4
5,NaN,B5,NaN,NaN,F5,G5
,A,B,C,D,F,G
0,A0,B0,C0,D0,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN
2,A2,B2,C2,D2,F2,G2


#### Many to One Operation and Set Logic Variants

The **one-to-many operation** `(1:n)` occurs when one row from left can be 
linked to several rows from right (linked through the key). 

*e.g*: one user can have different addresses.

**NOTE:** The reversed case will be **many-to-one**, but it shared the same behavior.

<img src="./assets/imgs/merge_one_to_many.png" width="600"/>

**NOTE:** In the combination step, `merge()` will broadacast values in the result from left to each row merged from right.

**NOTE:** `outer`, `left`, `right` practically will take the result from `inner` and just add the other rows in order (left first) filling missing spaces with NaNs.

In [60]:
# checking one to many merge()
left = pd.DataFrame(
    {
        "A": letter_list("A",4),
        "B": letter_list("B",4),
        "C": letter_list("C",4),
        "D": letter_list("D",4)
    }
)
right = pd.DataFrame(
    {
        "B": ["B0", "B0", "B1", "B5"],
        "F": letter_list("F",4),
        "G": letter_list("G",4),
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 
 
 
 
 B 
 F 
 G 
 
 
 
 
 0 
 B0 
 F0 
 G0 
 
 
 1 
 B0 
 F1 
 G1 
 
 
 2 
 B1 
 F2 
 G2 
 
 
 3 
 B5 
 F3 
 G3

In [61]:
# one to many with how = "inner" (interception of left and right keys)
# NOTE: how values from left are broadcasted to each row merged from right
pd.merge(left, right, on="B", how="inner")

,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A0,B0,C0,D0,F1,G1
2,A1,B1,C1,D1,F2,G2


In [62]:
# one to many with how = "outer" (union of left and right keys)
# NOTE: the other rows are just added with missing spaces = NaNs
outer_df = pd.merge(left, right, on="B", how="outer")
left_outer = pd.merge(left, right, on="B", how="left")
right_outer = pd.merge(left, right, on="B", how="right")

display_side_by_side(outer_df, left_outer, right_outer)

,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A0,B0,C0,D0,F1,G1
2,A1,B1,C1,D1,F2,G2
3,A2,B2,C2,D2,NaN,NaN
4,A3,B3,C3,D3,NaN,NaN
5,NaN,B5,NaN,NaN,F3,G3
,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A0,B0,C0,D0,F1,G1
2,A1,B1,C1,D1,F2,G2


#### Many to Many and Set Logic Variants

The **many-to-many operation** `(n:n)` occurs when several rows from left can be 
linked to several rows from right (linked through the key). 

*e.g*: one user can have several skills, but an skill also can be associated to several users.

<img src="./assets/imgs/merge_many_to_many.png" width="600"/>

**NOTE:** A **cross combination** occurs between rows with the same key. Where first the left rows are processed and then the right rows.

**NOTE:** Similar to `one-to-many`, the outer operations will just add the rest rows in order (left first) filling missing spaces with NaNs.

**NOTE:** Joining/merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large `DataFrames`.

In [63]:
# checking one to many merge()
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B":["B0", "B0", "B1", "B1", "B2"],
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    }
)
right = pd.DataFrame(
    {
        "B": ["B0", "B0", "B1", "B5"],
        "F": letter_list("F",4),
        "G": letter_list("G",4),
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B0 
 C1 
 D1 
 
 
 2 
 A2 
 B1 
 C2 
 D2 
 
 
 3 
 A3 
 B1 
 C3 
 D3 
 
 
 4 
 A4 
 B2 
 C4 
 D4 
 
 
 
 
 
 
 B 
 F 
 G 
 
 
 
 
 0 
 B0 
 F0 
 G0 
 
 
 1 
 B0 
 F1 
 G1 
 
 
 2 
 B1 
 F2 
 G2 
 
 
 3 
 B5 
 F3 
 G3

In [64]:
# NOTE: how the keys between both left and right are cross combined
# first left keys are taken and compare with each coincidence in the right
pd.merge(left, right, on="B", how="inner")

,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A0,B0,C0,D0,F1,G1
2,A1,B0,C1,D1,F0,G0
3,A1,B0,C1,D1,F1,G1
4,A2,B1,C2,D2,F2,G2
5,A3,B1,C3,D3,F2,G2


In [65]:
outer_df = pd.merge(left, right, on="B", how="outer")
left_outer = pd.merge(left, right, on="B", how="left")
right_outer = pd.merge(left, right, on="B", how="right")

display_side_by_side(outer_df, left_outer, right_outer)

,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A0,B0,C0,D0,F1,G1
2,A1,B0,C1,D1,F0,G0
3,A1,B0,C1,D1,F1,G1
4,A2,B1,C2,D2,F2,G2
5,A3,B1,C3,D3,F2,G2
6,A4,B2,C4,D4,NaN,NaN
7,NaN,B5,NaN,NaN,F3,G3
,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0


### Overlapping Value Columns

The **value columns** are the columns that are not keys. There are cases that 
we could have the same value columns from both left and right. In those cases,
the `merge()` automatically add suffixes to those columns in the result, 
indicating the source of the column.

**NOTE:** Commonly, the suffixes are `_x`, `_y`, ..., but you can configure them using `suffixes` argument.

In [74]:
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    }
)
right = pd.DataFrame(
    {
        "B": letter_list("B",4),
        "C": letter_list("c",4),
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 B 
 C 
 
 
 
 
 0 
 B0 
 c0 
 
 
 1 
 B1 
 c1 
 
 
 2 
 B2 
 c2 
 
 
 3 
 B3 
 c3

In [72]:
# NOTE: the value column repeated in both object is the column C, and 
# consequently suffixes are added to differentiate them.
result = pd.merge(left, right, on="B")
display_side_by_side(left, right, result)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 B 
 C 
 
 
 
 
 0 
 B0 
 c0 
 
 
 1 
 B1 
 c1 
 
 
 2 
 B2 
 c2 
 
 
 3 
 B3 
 c3 
 
 
 
 
 
 
 A 
 B 
 C_x 
 D 
 C_y 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 c0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 c1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 c2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 c3

In [73]:
# Setting the suffixes names
pd.merge(left, right, on="B", suffixes=("_l", "_r"))

,A,B,C_l,D,C_r
0,A0,B0,C0,D0,c0
1,A1,B1,C1,D1,c1
2,A2,B2,C2,D2,c2
3,A3,B3,C3,D3,c3


### Non-matching Names on Columns

There are cases where the column used as **key** doesn't have exactly the same 
name on both `left` and `right` objects.

In those cases, you can use `left_on` and `right_on` attributes to specified the names
of the keys on left and right respectively.

**NOTE**: both left and right keys will appear in the result.

In [75]:
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    }
)
right = pd.DataFrame(
    {
        "B_other": letter_list("B",4),
        "E": letter_list("F",4),
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 B_other 
 E 
 
 
 
 
 0 
 B0 
 F0 
 
 
 1 
 B1 
 F1 
 
 
 2 
 B2 
 F2 
 
 
 3 
 B3 
 F3

In [83]:
# Merging with non-matching column names
# NOTE: both left and right keys will appear in the result.
pd.merge(left, right, left_on="B", right_on="B_other")

,A,B,C,D,B_other,E
0,A0,B0,C0,D0,B0,F0
1,A1,B1,C1,D1,B1,F1
2,A2,B2,C2,D2,B2,F2
3,A3,B3,C3,D3,B3,F3


### Multiple, Index, and Index-Columns Keys


#### Multiple Keys

You can use more than one column as keys to `merge`. To do that, you have to 
use the argument `on = <list_of_columns>`. 

**NOTE:** You can do the same with `left_on` or `right_on`.

In [103]:
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    }
)

right = pd.DataFrame(
    {   
        "A": letter_list("A",4),
        "B": letter_list("B",4),
        "E": letter_list("F",4),
    }
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 A 
 B 
 E 
 
 
 
 
 0 
 A0 
 B0 
 F0 
 
 
 1 
 A1 
 B1 
 F1 
 
 
 2 
 A2 
 B2 
 F2 
 
 
 3 
 A3 
 B3 
 F3

In [104]:
pd.merge(left, right, on=["A","B"])

,A,B,C,D,E
0,A0,B0,C0,D0,F0
1,A1,B1,C1,D1,F1
2,A2,B2,C2,D2,F2
3,A3,B3,C3,D3,F3


#### Index Keys

You can use the index (or multi-index) as keys instead of columns. To do that,
you have to use `left_index = True` or `right_index = True`.

**NOTE:** When you use `left_index` or `right_index` **all the levels** on left
or right index will be used as keys, and preserved in the result.

**NOTE:** This use case is so common that the `join()` function was created practically as a sugar syntax of this cases.

In [105]:
left_index = pd.MultiIndex.from_arrays(
            [letter_list("X",5), letter_list("Y",5)],
            names=["X","Y"])
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index = left_index
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4)],
            names=["X","Y"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3

In [106]:
result = pd.merge(left, right, left_index=True, right_index=True)
display_side_by_side(left, right, result)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 F0 
 E0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 F1 
 E1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 F2 
 E2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 F3 
 E3

#### Index-Column Keys

You can also use a combination of index (multi-index) levels and columns as keys.

Examples:

1. All the index from left (using `left_index = True`) and columns from right (using `right_on`). It preserves the right index.
2. Columns from left (using `left_on`), and all the index from right (using `right_index = True`). It preserves the left index.
3. Combinations of both index levels and column in both objects (using `on`). Note the index levels must be named.

**NOTE:** In the cases where you use `on`, you can also use the variations
`left_on` and `right_on`.

In [107]:
left_index = pd.MultiIndex.from_arrays(
            [letter_list("X",5), letter_list("Y",5)],
            names=["X","Y"])
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index = left_index
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("A",4), letter_list("B",4)],
            names=["A","B"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "X": letter_list("X",4),
        "Y": letter_list("Y",4),
    },
    index = right_index
)

display_side_by_side(left, right)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 X 
 Y 
 
 
 A 
 B 
 
 
 
 
 
 
 
 A0 
 B0 
 F0 
 X0 
 Y0 
 
 
 A1 
 B1 
 F1 
 X1 
 Y1 
 
 
 A2 
 B2 
 F2 
 X2 
 Y2 
 
 
 A3 
 B3 
 F3 
 X3 
 Y3

In [109]:
# 1. all index left + columns on right
# NOTE: the index levels on right are preserved.
result = pd.merge(left, right, left_index=True, right_on=["X","Y"])
display_side_by_side(left, right, result)


A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 X 
 Y 
 
 
 A 
 B 
 
 
 
 
 
 
 
 A0 
 B0 
 F0 
 X0 
 Y0 
 
 
 A1 
 B1 
 F1 
 X1 
 Y1 
 
 
 A2 
 B2 
 F2 
 X2 
 Y2 
 
 
 A3 
 B3 
 F3 
 X3 
 Y3 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 F 
 X 
 Y 
 
 
 A 
 B 
 
 
 
 
 
 
 
 
 
 
 
 A0 
 B0 
 A0 
 B0 
 C0 
 D0 
 F0 
 X0 
 Y0 
 
 
 A1 
 B1 
 A1 
 B1 
 C1 
 D1 
 F1 
 X1 
 Y1 
 
 
 A2 
 B2 
 A2 
 B2 
 C2 
 D2 
 F2 
 X2 
 Y2 
 
 
 A3 
 B3 
 A3 
 B3 
 C3 
 D3 
 F3 
 X3 
 Y3

In [110]:
# 2. columns on left + all index right
# NOTE: the index levels on left are preserved.
result = pd.merge(left, right, right_index=True, left_on=["A","B"])
display_side_by_side(left, right, result)


A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 X 
 Y 
 
 
 A 
 B 
 
 
 
 
 
 
 
 A0 
 B0 
 F0 
 X0 
 Y0 
 
 
 A1 
 B1 
 F1 
 X1 
 Y1 
 
 
 A2 
 B2 
 F2 
 X2 
 Y2 
 
 
 A3 
 B3 
 F3 
 X3 
 Y3 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 F 
 X 
 Y 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 F0 
 X0 
 Y0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 F1 
 X1 
 Y1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 F2 
 X2 
 Y2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 F3 
 X3 
 Y3

In [112]:
# 3. index - column combination on both sides
# NOTE: the index levels must be named to use it.
result = pd.merge(left, right, on=["Y","A"])
display_side_by_side(left, right, result)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 X 
 Y 
 
 
 A 
 B 
 
 
 
 
 
 
 
 A0 
 B0 
 F0 
 X0 
 Y0 
 
 
 A1 
 B1 
 F1 
 X1 
 Y1 
 
 
 A2 
 B2 
 F2 
 X2 
 Y2 
 
 
 A3 
 B3 
 F3 
 X3 
 Y3 
 
 
 
 
 
 
 Y 
 A 
 B 
 C 
 D 
 F 
 X 
 
 
 
 
 0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 F0 
 X0 
 
 
 1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 F1 
 X1 
 
 
 2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 F2 
 X2 
 
 
 3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 F3 
 X3

### Extras: `indicator` and `validator` arguments

## Merging: `join()`

`left.join(right)` is also a merging operation similar to `merge()`, but `join()` will only 
perform a joining operation using row labels (`index`) by default.

However, there is also an optional attribute `on`, which allow us to pick a combination
of index levels or columns **on the left object**.

In that way, `join()` will always use 
- all index levels on right, and 
- all index levels, some columns, or some columns/index combination on left.

We can distinguish the following cases:

1. Joining on all `index` levels from right and some columns on left
2. Joining on all `index` levels from right and some combination columns/index on left
3. Joining on all `index` levels from right and all index levels on left.
    - when left index is a subset of right index.
    - when right index is a subset of left index.

**NOTE:** In the the cases 1 and 2, we will use `on`, which has contain the same number of keys as index levels on rigth.

Notice the important attribute of this method are:

`left.join(other, on = None, how = 'left')`

- `other` is the right other DataFrame or named Series object. Note the index (or multi-index) must be similar to columns or index in the first one.
- `how` (by default `left`) controls which index labels to keep in the result. It can be `left`, `right`, `outer`, `inner`, `cross`.
- `on` defines column or index level names of the `left` object.

**NOTE:** You can use `join` only as `df1.join(df2)` where df2 is the other.

**NOTE:** For many-to-one joins (where one of the DataFrame’s is already indexed by the join key), using join may be more convenient.

[TODO]: Make a picture

In [126]:
left = pd.DataFrame(
    {
        "X": letter_list("X",5),
        "Y": letter_list("Y",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    }
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4)],
            names=["X","Y"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)


# 1. join some columns from left and all index from right
# NOTE: on must take a list of the same size as multi levels on rigth
# NOTE: both X and Y are columns
result = left.join(right, on=["X", "Y"])

display_side_by_side(left, right, result)

# NOTE: the previous is less verbose than the merge variant
# pd.merge(left, right, left_on=["X", "Y"], right_index=True, how="left", sort=False)

X 
 Y 
 C 
 D 
 
 
 
 
 0 
 X0 
 Y0 
 C0 
 D0 
 
 
 1 
 X1 
 Y1 
 C1 
 D1 
 
 
 2 
 X2 
 Y2 
 C2 
 D2 
 
 
 3 
 X3 
 Y3 
 C3 
 D3 
 
 
 4 
 X4 
 Y4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3 
 
 
 
 
 
 
 X 
 Y 
 C 
 D 
 F 
 E 
 
 
 
 
 0 
 X0 
 Y0 
 C0 
 D0 
 F0 
 E0 
 
 
 1 
 X1 
 Y1 
 C1 
 D1 
 F1 
 E1 
 
 
 2 
 X2 
 Y2 
 C2 
 D2 
 F2 
 E2 
 
 
 3 
 X3 
 Y3 
 C3 
 D3 
 F3 
 E3 
 
 
 4 
 X4 
 Y4 
 C4 
 D4 
 NaN 
 NaN

In [127]:
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "Y": letter_list("Y",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index=pd.Index(letter_list("X",5), name="X")
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4)],
            names=["X","Y"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)


# 2. join some index/columns combination from left and all index from right
# NOTE: on must take a list of the same size as multi levels on rigth
# NOTE: X is a level of index and Y is a column
result = left.join(right, on=["X", "Y"])

display_side_by_side(left, right, result)

A 
 Y 
 C 
 D 
 
 
 X 
 
 
 
 
 
 
 
 
 X0 
 A0 
 Y0 
 C0 
 D0 
 
 
 X1 
 A1 
 Y1 
 C1 
 D1 
 
 
 X2 
 A2 
 Y2 
 C2 
 D2 
 
 
 X3 
 A3 
 Y3 
 C3 
 D3 
 
 
 X4 
 A4 
 Y4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3 
 
 
 
 
 
 
 A 
 Y 
 C 
 D 
 F 
 E 
 
 
 X 
 
 
 
 
 
 
 
 
 
 
 X0 
 A0 
 Y0 
 C0 
 D0 
 F0 
 E0 
 
 
 X1 
 A1 
 Y1 
 C1 
 D1 
 F1 
 E1 
 
 
 X2 
 A2 
 Y2 
 C2 
 D2 
 F2 
 E2 
 
 
 X3 
 A3 
 Y3 
 C3 
 D3 
 F3 
 E3 
 
 
 X4 
 A4 
 Y4 
 C4 
 D4 
 NaN 
 NaN

In [129]:
left_index = pd.MultiIndex.from_arrays(
            [letter_list("X",5), letter_list("Y",5)],
            names=["X","Y"])

left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "Y": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index=left_index
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4)],
            names=["X","Y"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)


# 3. join all index from left and all index from right
result = left.join(right)

display_side_by_side(left, right, result)

# Less verbose that merge variant
# merge(left, right, left_index = True, right_index = True)

A 
 Y 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3 
 
 
 
 
 
 
 
 A 
 Y 
 C 
 D 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 F0 
 E0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 F1 
 E1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 F2 
 E2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 F3 
 E3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 NaN 
 NaN

In [130]:
left_index = pd.MultiIndex.from_arrays(
            [letter_list("X",5), letter_list("Y",5), letter_list("Z",5)],
            names=["X","Y","Z"])
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index = left_index
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4)],
            names=["X","Y"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)


# 4. join all index from left and all index from right, when right index is a subset
# of left index
# NOTE: the objects will be joined by the interception "X" and "Y"
result = left.join(right)

display_side_by_side(left, right, result)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 Z 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 Z0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 Z1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 Z2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 Z3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 Z4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 
 
 
 
 
 
 X0 
 Y0 
 F0 
 E0 
 
 
 X1 
 Y1 
 F1 
 E1 
 
 
 X2 
 Y2 
 F2 
 E2 
 
 
 X3 
 Y3 
 F3 
 E3 
 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 F 
 E 
 
 
 X 
 Y 
 Z 
 
 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 Z0 
 A0 
 B0 
 C0 
 D0 
 F0 
 E0 
 
 
 X1 
 Y1 
 Z1 
 A1 
 B1 
 C1 
 D1 
 F1 
 E1 
 
 
 X2 
 Y2 
 Z2 
 A2 
 B2 
 C2 
 D2 
 F2 
 E2 
 
 
 X3 
 Y3 
 Z3 
 A3 
 B3 
 C3 
 D3 
 F3 
 E3 
 
 
 X4 
 Y4 
 Z4 
 A4 
 B4 
 C4 
 D4 
 NaN 
 NaN

In [131]:
left_index = pd.MultiIndex.from_arrays(
            [letter_list("X",5), letter_list("Y",5)],
            names=["X","Y"])
left = pd.DataFrame(
    {
        "A": letter_list("A",5),
        "B": letter_list("B",5),
        "C": letter_list("C",5),
        "D": letter_list("D",5)
    },
    index = left_index
)

right_index = pd.MultiIndex.from_arrays(
            [letter_list("X",4), letter_list("Y",4), letter_list("Z",4)],
            names=["X","Y","Z"])
right = pd.DataFrame(
    {   
        "F": letter_list("F",4),
        "E": letter_list("E",4),
    },
    index = right_index
)


# 5. join all index from left and all index from right, when right index is a subset
# of left index
# NOTE: the objects will be joined by the interception "X" and "Y"
result = left.join(right)

display_side_by_side(left, right, result)

A 
 B 
 C 
 D 
 
 
 X 
 Y 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 A0 
 B0 
 C0 
 D0 
 
 
 X1 
 Y1 
 A1 
 B1 
 C1 
 D1 
 
 
 X2 
 Y2 
 A2 
 B2 
 C2 
 D2 
 
 
 X3 
 Y3 
 A3 
 B3 
 C3 
 D3 
 
 
 X4 
 Y4 
 A4 
 B4 
 C4 
 D4 
 
 
 
 
 
 
 
 
 F 
 E 
 
 
 X 
 Y 
 Z 
 
 
 
 
 
 
 X0 
 Y0 
 Z0 
 F0 
 E0 
 
 
 X1 
 Y1 
 Z1 
 F1 
 E1 
 
 
 X2 
 Y2 
 Z2 
 F2 
 E2 
 
 
 X3 
 Y3 
 Z3 
 F3 
 E3 
 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 F 
 E 
 
 
 X 
 Y 
 Z 
 
 
 
 
 
 
 
 
 
 
 X0 
 Y0 
 Z0 
 A0 
 B0 
 C0 
 D0 
 F0 
 E0 
 
 
 X1 
 Y1 
 Z1 
 A1 
 B1 
 C1 
 D1 
 F1 
 E1 
 
 
 X2 
 Y2 
 Z2 
 A2 
 B2 
 C2 
 D2 
 F2 
 E2 
 
 
 X3 
 Y3 
 Z3 
 A3 
 B3 
 C3 
 D3 
 F3 
 E3 
 
 
 X4 
 Y4 
 NaN 
 A4 
 B4 
 C4 
 D4 
 NaN 
 NaN

Things to take into account

- on, left_on, and right_on
- left_index, right_index, -> join

- mergin on columns and index using ???
- Extras: `validate` and `indicator` arguments
- Series and DataFrames MErging
- Overlapping value columns

The related `join()` method, uses merge internally for the index-on-index (by default) and column(s)-on-index join.

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="left")
display_side_by_side(left, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 NaN 
 NaN 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 3 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 4 
 K2 
 K1 
 A3 
 B3 
 NaN 
 NaN

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="right")
display_side_by_side(right, result)

key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 NaN 
 NaN 
 C3 
 D3

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="outer")
display_side_by_side(left, right, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 NaN 
 NaN 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 3 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 4 
 K2 
 K1 
 A3 
 B3 
 NaN 
 NaN 
 
 
 5 
 K2 
 K0 
 NaN 
 NaN 
 C3 
 D3

In [ ]:
result = pd.merge(left, right, how="cross")
display_side_by_side(left, right, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1_x 
 key2_x 
 A 
 B 
 key1_y 
 key2_y 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K0 
 K0 
 A0 
 B0 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K0 
 K0 
 A0 
 B0 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K0 
 K0 
 A0 
 B0 
 K2 
 K0 
 C3 
 D3 
 
 
 4 
 K0 
 K1 
 A1 
 B1 
 K0 
 K0 
 C0 
 D0 
 
 
 5 
 K0 
 K1 
 A1 
 B1 
 K1 
 K0 
 C1 
 D1 
 
 
 6 
 K0 
 K1 
 A1 
 B1 
 K1 
 K0 
 C2 
 D2 
 
 
 7 
 K0 
 K1 
 A1 
 B1 
 K2 
 K0 
 C3 
 D3 
 
 
 8 
 K1 
 K0 
 A2 
 B2 
 K0 
 K0 
 C0 
 D0 
 
 
 9 
 K1 
 K0 
 A2 
 B2 
 K1 
 K0 
 C1 
 D1 
 
 
 10 
 K1 
 K0 
 A2 
 B2 
 K1 
 K0 
 C2 
 D2 
 
 
 11 
 K1 
 K0 
 A2 
 B2 
 K2 
 K0 
 C3 
 D3 
 
 
 12 
 K2 
 K1 
 A3 
 B3 
 K0 
 K0 
 C0 
 D0 
 
 
 13 
 K2 
 K1 
 A3 
 B3 
 K1 
 K0 
 C1 
 D1 
 
 
 14 
 K2 
 K1 
 A3 
 B3 
 K1 
 K0 
 C2 
 D2 
 
 
 15 
 K2 
 K1 
 A3 
 B3 
 K2 
 K0 
 C3 
 D3

In [ ]:
ser = pd.Series(
    ["a", "b", "c", "d", "e", "f"],
    index=pd.MultiIndex.from_arrays(
        [["A", "B", "C"] * 2, [1, 2, 3, 4, 5, 6]], names=["Let", "Num"]
    ),
)
ser

Let  Num
A    1      a
B    2      b
C    3      c
A    4      d
B    5      e
C    6      f
dtype: object

In [ ]:
ser.reset_index()

,Let,Num,0
0,A,1,a
1,B,2,b
2,C,3,c
3,A,4,d
4,B,5,e
5,C,6,f


**Warning:** Joining / merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large DataFrames.

In [ ]:
left = pd.DataFrame({"A": [1, 2], "B": [2, 2]})

right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2]})

result = pd.merge(left, right, on="B", how="outer")

display_side_by_side(left, right, result)

# este ejercicio me hace dudar todo lo que entendi antes

A 
 B 
 
 
 
 
 0 
 1 
 2 
 
 
 1 
 2 
 2 
 
 
 
 
 
 
 A 
 B 
 
 
 
 
 0 
 4 
 2 
 
 
 1 
 5 
 2 
 
 
 2 
 6 
 2 
 
 
 
 
 
 
 A_x 
 B 
 A_y 
 
 
 
 
 0 
 1 
 2 
 4 
 
 
 1 
 1 
 2 
 5 
 
 
 2 
 1 
 2 
 6 
 
 
 3 
 2 
 2 
 4 
 
 
 4 
 2 
 2 
 5 
 
 
 5 
 2 
 2 
 6

In [ ]:

result = pd.merge(left, right, on="B", how="inner")

display_side_by_side(left, right, result)

A 
 B 
 
 
 
 
 0 
 1 
 2 
 
 
 1 
 2 
 2 
 
 
 
 
 
 
 A 
 B 
 
 
 
 
 0 
 4 
 2 
 
 
 1 
 5 
 2 
 
 
 2 
 6 
 2 
 
 
 
 
 
 
 A_x 
 B 
 A_y 
 
 
 
 
 0 
 1 
 2 
 4 
 
 
 1 
 1 
 2 
 5 
 
 
 2 
 1 
 2 
 6 
 
 
 3 
 2 
 2 
 4 
 
 
 4 
 2 
 2 
 5 
 
 
 5 
 2 
 2 
 6

If a string matches both a column name and an index level name, then a warning is issued and the column takes precedence. This will result in an ambiguity error in a future version.

**Note**: the non-unique keys problem that I didn't understand happens because
there are overlapping cases where for example

![image.png](attachment:image.png)

If we want to 'inner' merge using k as key we encounter overlapping in the values
of v because

1. left has K0 1
2. right has K0 3 and K0 4

Then, the overlapping is solve by adding two columns at the resultin merged df
as v_x and v_y

v_x for the left values of column v and v_y for the right values of column v

In [ ]:
df1 = pd.DataFrame({"v": [1, 2, 3]}, index=["K0", "K1", "K2"])
df2 = pd.DataFrame({"v": [4, 5, 6]}, index=["K0", "K0", "K3"])
df3 = pd.DataFrame({"v": [7, 8, 9]}, index=["K1", "K1", "K2"])

result = df1.join([df2, df3])

display_side_by_side(df1, df2, df3, result)


,v
K0,1
K1,2
K2,3
,v
K0,4
K0,5
K3,6
,v
K1,7
K1,8


In [ ]:
#the idea of combine_first is to have two data frames that are similar in the sense
# that they have same index and columns (not necessarily all of them)
# and you want to "patch" or fill some nan values from left with the values
# from right.

df1 = pd.DataFrame(
    [[np.nan, 3.0, 5.0], [-4.6, np.nan, np.nan], [np.nan, 7.0, np.nan]]
)


df2 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5.0, 1.6, 4]], index=[1, 2])

result = df1.combine_first(df2)

display_side_by_side(df1, df2, result)

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,NaN
2,NaN,7.0,NaN
,0,1,2
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0
,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,-8.2
2,-5.0,7.0,4.0


In [ ]:
# related method that alterate the NaN values in place
# but replace all the values from right into the left structure
df1.update(df2)
df1

,0,1,2
0,NaN,3.0,5.0
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0


In [ ]:
# how = "cross" (cartesian product between rows in left and right)
# NOTE: it is not one to one, it is a cartesian product
pd.merge(left, right, how="cross")